In [1]:
import argparse
import sys
import os
import json
import numpy as np
from osgeo import gdal, osr
from shapely.geometry import shape, mapping
import pandas as pd
import geopandas as gpd
from shapely import wkt
import glob
import matplotlib.pyplot as plt
import json, requests, os
from datetime import datetime
import rasterio
import pyodbc
import database.Database as Database
import pytz
import database.credentialsDB as credentialsDB
from pathlib import Path
import database.credentialsDB as  accessDB
from psycopg2 import sql
import psycopg2
import extract_md as ex
from dotenv import load_dotenv
import os
import change_path_to_url as ptu

In [4]:
# Load CSV file into a Pandas DataFrame
csv_file = '../data-3857.csv'  # replace with your CSV file path
data = pd.read_csv(csv_file)

# Convert the 'st_astext' column to geometries
data['geometry'] = data['st_astext'].apply(wkt.loads)

# Filter rows where 'cc' is less than 30
filtered_data = data[data['cc'] < 30]

# Convert geometries to well-known text (WKT) and then back to ensure compatibility
filtered_data['geometry'] = filtered_data['geometry'].apply(lambda geom: wkt.loads(wkt.dumps(geom)))

# Create a GeoDataFramepi
gdf = gpd.GeoDataFrame(filtered_data, geometry='geometry')

# Save the GeoDataFrame to a GeoJSON file
geojson_file_path = 's2_polska_2023_2024_wkt.geojson'
gdf.to_file(geojson_file_path, driver='GeoJSON')

print(f"GeoJSON file saved to: {geojson_file_path}")

/tmp/ipykernel_961188/2390208326.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['geometry'] = filtered_data['geometry'].apply(lambda geom: wkt.loads(wkt.dumps(geom)))


GeoJSON file saved to: s2_polska_2023_2024_wkt.geojson


In [5]:
# Extract the path_list from the 'product_identifier' column
path_list = filtered_data['product_identifier'].tolist()

record_count = len(path_list)
record_count

path_list=path_list[:20]
path_list

['/eodata/Sentinel-2/MSI/L2A/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T33UYR_20230102T113531.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/01/03/S2A_MSIL2A_20230103T101411_N0509_R022_T33UVA_20230103T141354.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/01/03/S2A_MSIL2A_20230103T101411_N0509_R022_T33UWV_20230103T141354.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/01/03/S2A_MSIL2A_20230103T101411_N0509_R022_T33UVT_20230103T141354.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/01/14/S2A_MSIL2A_20230114T094341_N0509_R036_T34UGB_20230114T140700.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/01/16/S2A_MSIL2A_20230116T102351_N0509_R065_T33UVV_20230116T135854.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/03/01/S2A_MSIL2A_20230301T100021_N0509_R122_T33UWV_20230301T142802.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/03/01/S2A_MSIL2A_20230301T100021_N0509_R122_T34UCA_20230301T142802.SAFE',
 '/eodata/Sentinel-2/MSI/L2A

In [6]:
#to ponizej puszczacie w termianlu i wyciagacie swoje klucze do eodaty, ktore pozniej wklejacie w miejscce
# curl http://169.254.169.254/openstack/latest/vendor_data2.json

# Wczytaj zmienne środowiskowe z pliku .env
load_dotenv()

# Teraz zmienne środowiskowe są dostępne i możesz je ustawić w kodzie
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')

# Pozostałe zmienne
os.environ['GDAL_HTTP_TCP_KEEPALIVE'] = "YES"
os.environ['AWS_S3_ENDPOINT'] = "eodata.cloudferro.com"
os.environ['AWS_HTTPS'] = "YES"
os.environ['AWS_VIRTUAL_HOSTING'] = "FALSE"

In [7]:
import subprocess
def getFullPath(var):
    # Run the s5cmd command and capture the output
    command = ['s5cmd', '--endpoint-url', 'https://eodata.cloudferro.com', 'ls',
               's3:/' + var + '/GRANULE/*/IMG_DATA/R20m/*.jp2']
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, error = process.communicate()
    # Check if there was any error
    if process.returncode != 0:
        print(f"Error: {error.decode('utf-8')}")
        return None
    else:
        # Parse the output and extract the relative paths
        output_text = output.decode('utf-8')
        lines = output_text.split('\n')
        relative_paths = [line.split()[-1] for line in lines if line.strip()]
        return [var + '/GRANULE/' + path for path in relative_paths]

In [8]:
for directory_path in path_list:
    print(f"Contents of {directory_path}:")
    
    matching_directories = glob.glob(os.path.join(directory_path, "GRANULE/*/IMG_DATA/R20m"))

    for directory in matching_directories:
        for root, dirs, files in os.walk(directory):
            for file in files:
                print(os.path.join(root, file))

Contents of /eodata/Sentinel-2/MSI/L2A/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE:
/eodata/Sentinel-2/MSI/L2A/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE/GRANULE/L2A_T34UCA_A030421_20230102T095318/IMG_DATA/R20m/T34UCA_20230102T095319_AOT_20m.jp2
/eodata/Sentinel-2/MSI/L2A/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE/GRANULE/L2A_T34UCA_A030421_20230102T095318/IMG_DATA/R20m/T34UCA_20230102T095319_B01_20m.jp2
/eodata/Sentinel-2/MSI/L2A/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE/GRANULE/L2A_T34UCA_A030421_20230102T095318/IMG_DATA/R20m/T34UCA_20230102T095319_B02_20m.jp2
/eodata/Sentinel-2/MSI/L2A/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE/GRANULE/L2A_T34UCA_A030421_20230102T095318/IMG_DATA/R20m/T34UCA_20230102T095319_B03_20m.jp2
/eodata/Sentinel-2/MSI/L2A/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.

In [9]:
import os
import datetime
import numpy as np
import rasterio
from rasterio.io import MemoryFile
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
from rasterio.crs import CRS
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Define a continuous colormap with 256 unique colors
spectral_colormap = plt.get_cmap('gist_earth', 256)

# Generate the colormap for 256 values
se2waq_colormap = {i: tuple(int(c*255) for c in spectral_colormap(i)[:3]) for i in range(256)}

def extract_date_and_tile_id(identifier):
    try:
        date_part = "/".join(identifier.split('/')[5:8])
        date_obj = datetime.datetime.strptime(date_part, '%Y/%m/%d')
        tile_id = identifier.split('_')[-2]
        return date_obj.year, date_obj.month, date_obj.day, tile_id
    except Exception as e:
        print(f"Error extracting date and tile ID from identifier {identifier}: {e}")
        return None, None, None, None

output_base_path = '../output_se2waq_doc/'
os.makedirs(output_base_path, exist_ok=True)
os.chmod(output_base_path, 0o777)

se2waq_list = []

for directory_path in path_list:
    print(f"Calculating Se2WaQ for {directory_path}:")

    matching_directories = getFullPath(directory_path)
    identifier = directory_path
    year, month, day, tile_id = extract_date_and_tile_id(identifier)
    
    if year is None or month is None or day is None:
        print(f"Skipping directory {directory_path} due to missing date or tile information.")
        continue

    # Add B02 to the list of bands
    subs = ["B01_20m.jp2", "B02_20m.jp2", "B03_20m.jp2", "B04_20m.jp2", "SCL_20m.jp2"]
    b01_band_path, b02_band_path, b03_band_path, b04_band_path, scl_path = list(filter(lambda x: any(sub in x for sub in subs), matching_directories))

    print(b01_band_path)

    with rasterio.open(f's3:/{b01_band_path}') as b01_src, \
         rasterio.open(f's3:/{b02_band_path}') as b02_src, \
         rasterio.open(f's3:/{b03_band_path}') as b03_src, \
         rasterio.open(f's3:/{b04_band_path}') as b04_src, \
         rasterio.open(f's3:/{scl_path}') as scl_src:

        # Read the bands
        b01_band = b01_src.read(1).astype(float)
        b02_band = b02_src.read(1).astype(float)
        b03_band = b03_src.read(1).astype(float)
        b04_band = b04_src.read(1).astype(float)

        # Reproject all bands to a common CRS and resolution
        dst_crs = CRS.from_epsg(3857)
        transform, width, height = calculate_default_transform(
            b03_src.crs, dst_crs, b03_src.width, b03_src.height, *b03_src.bounds)

        # Initialize arrays for resampling
        scl_resampled = np.empty((height, width), dtype=scl_src.read(1).dtype)
        b01_resampled = np.empty((height, width), dtype=np.float32)
        b02_resampled = np.empty((height, width), dtype=np.float32)
        b03_resampled = np.empty((height, width), dtype=np.float32)
        b04_resampled = np.empty((height, width), dtype=np.float32)

        # Reproject SCL band (water mask)
        reproject(
            source=scl_src.read(1),
            destination=scl_resampled,
            src_transform=scl_src.transform,
            src_crs=scl_src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest
        )

        # Reproject B01, B02, B03, B04 bands
        reproject(
            source=b01_band,
            destination=b01_resampled,
            src_transform=b01_src.transform,
            src_crs=b01_src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.bilinear
        )

        reproject(
            source=b02_band,
            destination=b02_resampled,
            src_transform=b02_src.transform,
            src_crs=b02_src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.bilinear
        )

        reproject(
            source=b03_band,
            destination=b03_resampled,
            src_transform=b03_src.transform,
            src_crs=b03_src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.bilinear
        )

        reproject(
            source=b04_band,
            destination=b04_resampled,
            src_transform=b04_src.transform,
            src_crs=b04_src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.bilinear
        )

        # Create the water mask based on the SCL band
        water_mask = (scl_resampled == 6)
        se2waq = np.full_like(b03_resampled, 255, dtype='uint8')

        # Calculate DOC index
        doc_water = 432 * np.exp(-2.24 * (b03_resampled[water_mask] / b04_resampled[water_mask]))

        # Normalize the DOC values
        doc_min, doc_max = doc_water.min(), doc_water.max()
        se2waq_normalized = (np.sqrt(doc_water - doc_min) / np.sqrt(doc_max - doc_min)) * 255
        se2waq_normalized = np.clip(se2waq_normalized, 0, 255).round().astype('uint8')

        # Enhance the color range for values between 0 and 25
        #low_values_mask = (se2waq_normalized >= 0) & (se2waq_normalized <= 25)
        #se2waq_normalized[low_values_mask] = (se2waq_normalized[low_values_mask] / 25.0 * 255).round().astype('uint8')

        # Update the water mask with normalized DOC values
        se2waq[water_mask] = se2waq_normalized
        se2waq_list.append(se2waq)

        # Prepare metadata for the output file
        kwargs = b03_src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height,
            'driver': 'GTiff',
            'dtype': 'uint8',
            'count': 1,
            'nodata': 255,
            'compress': 'zstd',
        })

        config = {
            "GDAL_NUM_THREADS": "ALL_CPUS",
            "GDAL_TIFF_INTERNAL_MASK": True,
            "GDAL_TIFF_OVR_BLOCKSIZE": "256",
            "BIGTIFF": "YES",
            "SPARSE_OK": "TRUE",
            "COMPRESS": "ZSTD",
        }

        # Write the result to a COG file
        with MemoryFile() as memfile:
            with memfile.open(**kwargs) as dst:
                dst.write(se2waq, 1)
                dst.write_colormap(1, se2waq_colormap)

            timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
            unique_id = f"{os.path.basename(directory_path)}_{timestamp}_Se2WaQ"
            date_path = os.path.join(output_base_path, f"{year}/{month:02d}/{day:02d}")
            os.makedirs(date_path, exist_ok=True)

            output_filename = os.path.join(date_path, f"{unique_id}.tif")

            cog_translate(
                memfile.name,
                output_filename,
                cog_profiles.get("deflate"),
                config=config,
                in_memory=True,
                quiet=False
            )

            print(f"Saved COG to {output_filename}")


Calculating Se2WaQ for /eodata/Sentinel-2/MSI/L2A/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE:
/eodata/Sentinel-2/MSI/L2A/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE/GRANULE/L2A_T34UCA_A030421_20230102T095318/IMG_DATA/R20m/T34UCA_20230102T095319_B01_20m.jp2


Reading input: /vsimem/b56e96a7-8241-4547-b502-14485e31c1cb/b56e96a7-8241-4547-b502-14485e31c1cb.tif

Adding overviews...
Updating dataset tags...
Writing output to: ../output_se2waq_doc/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE_20240906090631_Se2WaQ.tif


Saved COG to ../output_se2waq_doc/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE_20240906090631_Se2WaQ.tif
Calculating Se2WaQ for /eodata/Sentinel-2/MSI/L2A/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T33UYR_20230102T113531.SAFE:
/eodata/Sentinel-2/MSI/L2A/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T33UYR_20230102T113531.SAFE/GRANULE/L2A_T33UYR_A030421_20230102T095318/IMG_DATA/R20m/T33UYR_20230102T095319_B01_20m.jp2


Reading input: /vsimem/2dadb16a-fc1f-4da2-abda-a3be41b59b78/2dadb16a-fc1f-4da2-abda-a3be41b59b78.tif

Adding overviews...
Updating dataset tags...
Writing output to: ../output_se2waq_doc/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T33UYR_20230102T113531.SAFE_20240906090659_Se2WaQ.tif


Saved COG to ../output_se2waq_doc/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T33UYR_20230102T113531.SAFE_20240906090659_Se2WaQ.tif
Calculating Se2WaQ for /eodata/Sentinel-2/MSI/L2A/2023/01/03/S2A_MSIL2A_20230103T101411_N0509_R022_T33UVA_20230103T141354.SAFE:
/eodata/Sentinel-2/MSI/L2A/2023/01/03/S2A_MSIL2A_20230103T101411_N0509_R022_T33UVA_20230103T141354.SAFE/GRANULE/L2A_T33UVA_A039344_20230103T101412/IMG_DATA/R20m/T33UVA_20230103T101411_B01_20m.jp2


...

Saved COG to ../output_se2waq_doc/2023/03/01/S2A_MSIL2A_20230301T100021_N0509_R122_T33UWS_20230301T142802.SAFE_20240906091215_Se2WaQ.tif
Calculating Se2WaQ for /eodata/Sentinel-2/MSI/L2A/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UFE_20230228T110639.SAFE:
/eodata/Sentinel-2/MSI/L2A/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UFE_20230228T110639.SAFE/GRANULE/L2A_T34UFE_A031236_20230228T094031/IMG_DATA/R20m/T34UFE_20230228T094029_B01_20m.jp2


Reading input: /vsimem/9c57a60e-9718-4e3e-a36a-084365fee4c3/9c57a60e-9718-4e3e-a36a-084365fee4c3.tif

Adding overviews...
Updating dataset tags...
Writing output to: ../output_se2waq_doc/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UFE_20230228T110639.SAFE_20240906091243_Se2WaQ.tif


Saved COG to ../output_se2waq_doc/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UFE_20230228T110639.SAFE_20240906091243_Se2WaQ.tif
Calculating Se2WaQ for /eodata/Sentinel-2/MSI/L2A/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.SAFE:
/eodata/Sentinel-2/MSI/L2A/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.SAFE/GRANULE/L2A_T34UEE_A031236_20230228T094031/IMG_DATA/R20m/T34UEE_20230228T094029_B01_20m.jp2


Reading input: /vsimem/28dfdeb1-737b-4ac5-ac24-3d18ca78ec10/28dfdeb1-737b-4ac5-ac24-3d18ca78ec10.tif

Adding overviews...
Updating dataset tags...
Writing output to: ../output_se2waq_doc/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.SAFE_20240906091306_Se2WaQ.tif


Saved COG to ../output_se2waq_doc/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.SAFE_20240906091306_Se2WaQ.tif


In [11]:
#current_date = datetime.now()
formatted_date = datetime.now(pytz.timezone('Europe/Warsaw')).strftime("%Y-%m-%d %H:%M:%S")

# Construct the path to the credentials file relative to this file
credentialsDB_path = 'database/credentialsDB.txt'

credentialDB = accessDB.CredentialsDB(credentialsDB_path) #dla bazy danych

In [12]:
# Searching of tifs in location
import os

def find_tif_files(root_folder):
    tif_files = []
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('.tif'):
                tif_files.append(os.path.join(dirpath, filename))
    return tif_files

# Przykładowe użycie
root_folder = '/home/eouser/Desktop/geoportall/output_se2waq_doc/'
tif_files = find_tif_files(root_folder)

# Wyświetlanie znalezionych plików .tif
for tif_file in tif_files:
    print(tif_file)


/home/eouser/Desktop/geoportall/output_se2waq_doc/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.SAFE_20240906091306_Se2WaQ.tif
/home/eouser/Desktop/geoportall/output_se2waq_doc/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UFE_20230228T110639.SAFE_20240906091243_Se2WaQ.tif
/home/eouser/Desktop/geoportall/output_se2waq_doc/2023/01/14/S2A_MSIL2A_20230114T094341_N0509_R036_T34UGB_20230114T140700.SAFE_20240906090825_Se2WaQ.tif
/home/eouser/Desktop/geoportall/output_se2waq_doc/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE_20240906090631_Se2WaQ.tif
/home/eouser/Desktop/geoportall/output_se2waq_doc/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T33UYR_20230102T113531.SAFE_20240906090659_Se2WaQ.tif
/home/eouser/Desktop/geoportall/output_se2waq_doc/2023/01/16/S2A_MSIL2A_20230116T102351_N0509_R065_T33UVV_20230116T135854.SAFE_20240906090850_Se2WaQ.tif
/home/eouser/Desktop/geoportall/output_se2waq_doc/2023/01/03/S2A_MSIL2A_20230103T1

In [13]:
#Base path to the output directory

output_base_path = '/home/eouser/Desktop/geoportall/output_se2waq_doc/'

# Initialize a set to keep track of product IDs in the file paths
product_ids_in_paths = set()
liczba_tif = 0
# Iterate through all files in the output directory
for root, dirs, files in os.walk(output_base_path):
    for file in files:
        if file.endswith(".tif"):
            liczba_tif = liczba_tif +1
            file_path = os.path.join(root, file)
            parts = file_path.split(os.sep)
            product_id = parts[-1].replace(".SAFE", ".tif")
            product_ids_in_paths.add(product_id)

print(liczba_tif)
# Initialize a set to keep track of matching product IDs
matching_product_ids = set()

# Check for corresponding records in the GeoDataFrame
for product_id in product_ids_in_paths:
    if not gdf[gdf['product_identifier'].str.contains(product_id.replace(".tif", ".SAFE"))].empty:
        matching_product_ids.add(product_id)
    else:
        print(f"No matching record found for product ID: {product_id}")

# Log the number of matches and mismatches
print(f"Total product IDs in file paths: {len(product_ids_in_paths)}")
print(f"Total matching product IDs: {len(matching_product_ids)}")
print(f"Total missing product IDs: {len(product_ids_in_paths - matching_product_ids)}")

# Optional: Log missing product IDs
for missing_product_id in product_ids_in_paths - matching_product_ids:
    print(f"Missing product ID: {missing_product_id}")

20
No matching record found for product ID: S2A_MSIL2A_20230301T100021_N0509_R122_T33UVS_20230301T142802.tif_20240906091003_Se2WaQ.tif
No matching record found for product ID: S2A_MSIL2A_20230301T100021_N0509_R122_T34UCE_20230301T142802.tif_20240906091134_Se2WaQ.tif
No matching record found for product ID: S2A_MSIL2A_20230103T101411_N0509_R022_T33UVA_20230103T141354.tif_20240906090721_Se2WaQ.tif
No matching record found for product ID: S2A_MSIL2A_20230301T100021_N0509_R122_T33UWT_20230301T142802.tif_20240906091056_Se2WaQ.tif
No matching record found for product ID: S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.tif_20240906091306_Se2WaQ.tif
No matching record found for product ID: S2A_MSIL2A_20230116T102351_N0509_R065_T33UVV_20230116T135854.tif_20240906090850_Se2WaQ.tif
No matching record found for product ID: S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.tif_20240906090631_Se2WaQ.tif
No matching record found for product ID: S2B_MSIL2A_20230228T094029_N0509

In [14]:
import extract_md as ex
import change_path_to_url_doc as impdoc
# Połączenie do bazy
conn = Database.Database(
    credentialDB.host,
    credentialDB.dbname,
    credentialDB.username,
    credentialDB.password,
    credentialDB.port
)
conn.connect()

# Ustawienie autocommit na true
conn.connection.autocommit = True

# Utworzenie obiektu cursor
cursor = conn.connection.cursor()

# Sprawdzenie, czy rozszerzenie PostGIS jest zainstalowane
try:
    cursor.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
    print("PostGIS extension enabled.")
except Exception as e:
    print(f"Error enabling PostGIS extension: {e}")

# Dodanie przestrzennego indeksu, jeśli jeszcze nie istnieje
try:
    cursor.execute("""
        DO $$ 
        BEGIN
            IF NOT EXISTS (SELECT 1 FROM pg_indexes WHERE indexname = 'doc_geometry_idx') THEN
                CREATE INDEX doc_geometry_idx ON se2waq_doc USING GIST (geometry);
            END IF;
        END $$;
    """)
    print("Spatial index created or already exists.")
except Exception as e:
    print(f"Error creating spatial index: {e}")

# Ścieżka bazowa do katalogu wyjściowego
output_base_path = '/home/eouser/Desktop/geoportall/output_se2waq_doc/'

# Iteracja po wszystkich plikach w katalogu wyjściowym
for root, dirs, files in os.walk(output_base_path):
    for file in files:
        if file.endswith(".tif"):
            file_path = os.path.join(root, file)
            print(f"Processing file: {file_path}")  # Log the file being processed
            
            date_obj, product_id = ex.extract_metadata_from_path(file_path)
            
            if date_obj is None or product_id is None:
                print(f"Skipping file due to metadata extraction failure: {file_path}")
                continue  # Skip if the metadata extraction failed
            
            # Print extracted metadata for debugging
            print(f"Extracted metadata: date_obj={date_obj}, product_id={product_id}")




            # Get cloud cover and geometry from the GeoDataFrame
            try:
                tile_id = product_id.split('_')[-4]
                record = gdf[gdf['product_identifier'].str.contains(tile_id)]
                if record.empty:
                    print(f"No matching record found for product ID: {product_id}")
                    continue  # Skip if no matching record found

                cloud_cover = record['cc'].values[0]
                geometry = record['geometry'].iloc[0]  # Use .iloc[0] to get the first geometry

                print(f"Metadata extracted: product_id={product_id}, cloud_cover={cloud_cover}, geometry={geometry}")  # Log extracted metadata

                #Zmiana patha na link z gdala
                path_url = impdoc.change_path_to_url(file_path)
                #print(path_url)


                # Insert metadata into the database
                try:
                    insert_query = sql.SQL(
                        "INSERT INTO se2waq_doc (product_id, date, cloud_cover, file_path, geometry) VALUES (%s, %s, %s, %s, ST_GeomFromText(%s, 3857))"
                    )
                    conn.execute_query(insert_query, (product_id, date_obj, cloud_cover, path_url, geometry.wkt), f"File {product_id} inserted successfully to {file_path}")
                except Exception as e:
                    print(f"Error inserting record into the database for file {file_path}: {e}")

            except Exception as e:
                print(f"Error extracting cloud cover and geometry for file {file_path}: {e}")

# Zamknięcie połączenia z bazą danych
conn.close()


Połączenie z bazą danych powiodło się
PostGIS extension enabled.
Spatial index created or already exists.
Processing file: /home/eouser/Desktop/geoportall/output_se2waq_doc/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.SAFE_20240906091306_Se2WaQ.tif
Extracted metadata: date_obj=2023-02-28 00:00:00, product_id=S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.SAFE_20240906091306_Se2WaQ.tif
Metadata extracted: product_id=S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.SAFE_20240906091306_Se2WaQ.tif, cloud_cover=2.595227, geometry=POLYGON ((2337675.2205599537 7198255.485314934, 2337676.007474751 7012866.487615233, 2520447.4816926997 7010766.194705583, 2524763.4398823907 7196026.971770144, 2337675.2205599537 7198255.485314934))
Wysłano do bazy informację o kaflu nr File S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.SAFE_20240906091306_Se2WaQ.tif inserted successfully to /home/eouser/Desktop/geoportall/output_se2waq_doc/2023/0

In [2]:
# Synchronizacja plików z katalogu output_cog z bucketem S3
cog='s3cmd sync /home/eouser/Desktop/geoportall/output_se2waq_doc/ s3://SE2WAQ_DOC'

os.system(cog)


upload: '/home/eouser/Desktop/geoportall/output_se2waq_doc/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T33UYR_20230102T113531.SAFE_20240906090659_Se2WaQ.tif' -> 's3://SE2WAQ_DOC/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T33UYR_20230102T113531.SAFE_20240906090659_Se2WaQ.tif'  [1 of 21]
 932678 of 932678   100% in    0s    20.10 MB/s  done
upload: '/home/eouser/Desktop/geoportall/output_se2waq_doc/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE_20240906090631_Se2WaQ.tif' -> 's3://SE2WAQ_DOC/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE_20240906090631_Se2WaQ.tif'  [2 of 21]
 897524 of 897524   100% in    0s    26.70 MB/s  done
upload: '/home/eouser/Desktop/geoportall/output_se2waq_doc/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE_20240906090631_Se2WaQ.tif.aux.xml' -> 's3://SE2WAQ_DOC/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE_20240906090631_Se2WaQ.tif.au

0